In [38]:
import gensim
import numpy as np
import matplotlib.pyplot as plt
import random
import scipy as sp
import nltk
import jieba #中文分词
import time
import numpy.linalg as LA
import queue
import csv
import re
from sklearn.decomposition import PCA

In [4]:
jieba.load_userdict('/Users/tony/Desktop/OfflineClustering/RawData/dict-v24.txt')

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/27/ncssc1kj6nqgt3phh_0cfnn40000gn/T/jieba.cache
Loading model cost 0.641 seconds.
Prefix dict has been built succesfully.


KeyboardInterrupt: 

In [22]:
#hyperparas
DIMENSION = len(word_list)

## How to mess around with JIEBA:
1.分词 2.print individual words

In [48]:
seg_list = jieba.cut("市场走势")
print(seg_list)
for token in seg_list:
    print(token)
print("Full Mode: " + "/ ".join(seg_list))  # 全模式

<generator object Tokenizer.cut at 0x1a346d5a20>
市场走势
Full Mode: 


In [5]:

with open('/Users/tony/Desktop/OfflineClustering/RawData/datap_news_html.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',') ############分词符号是 ,
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            #print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            #print(f'\t{row[0]} works in the {row[1]} department, and was born in {row[2]}.')
            line_count += 1
        a = jieba.cut(row[2])
        print("///".join(a))
        
        if line_count > 10:
                  break
    print(f'Processed {line_count} lines.')

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/27/ncssc1kj6nqgt3phh_0cfnn40000gn/T/jieba.cache
Loading model cost 0.769 seconds.
Prefix dict has been built succesfully.


title
生意///社///：///本周///纯碱///市场走势///下行///（///12.11///-///12.15///）
阎庆民///到任///证监会/// ///分管///上市///部///、///投保///部///和///打非///局等/// 
李克强///：///希望///香港///努力///将///粤港澳///大湾///区///打///造成///国际///经济///竞争///格局///中///的///世界///一流///湾区
美国///税改///再遇///阻力/// ///两名///共和党///参议员///一///反对///一///犹疑
超///百家///企业///IPO///终止///审查/// ///理性分析///无需///恐慌
苏宁///减持///阿里///背后///：///备战///“///新///零售///”///下半场
美///零售///数据///强劲///提振///美元/// ///黄金///承压///震荡///走低
融资///余额///上涨///23.19///亿元/// /// ///融资///客抢筹///这///20///股///（///附///名单///）
黄金///日内///交易///分析///：///金价///围绕///50///日///均线///徘徊/// ///继续///下跌
生意///社///：///本周///磷矿石///市场///整体///持稳///，///局部///震荡///（///12.11///-///12.15///）
Processed 11 lines.


## Import Keys

In [15]:
word_list = []
with open('/Users/tony/Desktop/OfflineClustering/RawData/dict-v24.txt') as file:
    reader = csv.reader(file, delimiter = ' ')
    for row in reader:
        word_list.append(row[0])

word_set = set(word_list) 

word_id_dict = {}
for i in range(len(word_list)):
    word_id_dict[word_list[i]] = i 

Form the Key Set

## Data Cleansing

In [7]:
punctuation = '，：/？“”【】 ！。.\\*（）()’‘·、!,;:?"\''
def remove_punctuation(text):
    text = re.sub(r'[{}]+'.format(punctuation),'',text)
    return text

In [8]:
dic = {0:"零", 1:"一",2:"二",3:"三",4:"四",5:"五",6:"六",7:"七",8:"八",9:"九"}
target = ["1","2","3","4","5","6","7","8","9","0"]
def cleandigit(text):
    ret = ""
    for i in range(len(text)):
        if text[i] in target:
            ret += dic[int(text[i])]
        else:
            ret += text[i]
    return ret

## Prepare the News Data

In [28]:
data = []
with open('/Users/tony/Desktop/OfflineClustering/RawData/datap_news_html.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',') ##################### news must be separated by commas
    line_count = 0
    for row in csv_reader:
        line_count += 1
        #去除标点符号 改变数字形式
        tempstr = remove_punctuation(row[2])
        a = jieba.cut(cleandigit(tempstr))
        temp = []
        for token in a:
            temp.append(token)
        data.append(temp)
        
        if line_count > 1000: ############################### decide how many docs you wanna have experiments with
                  break
    print(f'Processed {line_count} lines.')

Processed 1001 lines.


In [71]:
print(data[1])
data = data[1:]
print(word_list[len(word_list) - 1])
print('一' in word_id_dict.keys())

['阎庆民', '到任', '证监会', '分管', '上市', '部', '投保', '部和', '打非', '局等']
15:15:70
False


## Build Dictionary demo

In [9]:
word2vec = {}
t = time.process_time()

with open("/Users/tony/Desktop/OfflineClustering/RawData/dict-v24.txt") as a, open("/Users/tony/Desktop/OfflineClustering/RawData/word2vec-v24.txt") as b:
    keys  = csv.reader(a, delimiter = ' ')
    values = csv.reader(b, delimiter = ' ')
    line_count = 0
    for key,val in zip(keys,values):
        word2vec[key[0]] = val[1:]
        line_count += 1
elapsed_time = time.process_time() - t
print(elapsed_time)

429.17013000000003


## Bag of Words(BOW)

In [23]:
def BOWwithSVD(data, method, dim = 300):#Using global word_set
    #Methods: 1.hash, hash to a fixed size; 2.SVD, pick out principal components
    doc_vecs = []
    if method == "hash": #only using mod right now
        for document in data:
            doc_vec = np.zeros(dim)
            for token in document:
                if token in word_id_dict.keys():
                    pos = word_id_dict[token] 
                    pos = pos % dim
                    doc_vec[pos] += 1
            doc_vecs.append(doc_vec)
    elif method == "SVD":
        for document in data:
            doc_vec = np.zeros(DIMENSION)
            for token in document:
                if token in word_id_dict.keys():
                    pos = word_id_dict[token] 
                    doc_vec[pos] += 1
            doc_vecs.append(doc_vec)
        pca = PCA(n_components=dim)
        doc_vecs = np.array(doc_vecs)
        doc_vecs = pca.fit(doc_vecs.T).T
    elif method == "naive":
        for document in data:
            doc_vec = np.zeros(DIMENSION)
            for token in document:
                if token in word_id_dict.keys():
                    pos = word_id_dict[token] 
                    doc_vec[pos] += 1
            doc_vecs.append(doc_vec)
    return doc_vecs

In [34]:
data = data[1:]
doc_vecs = BOWwithSVD(data, method = "naive")
print(len(doc_vecs))

995


## Cosine Similarity

In [40]:
def cos_similarity(doc1, doc2):
    if len(doc1) != len(doc2):
        return 0
    else:
        nom = np.dot(doc1, doc2)
        denom = LA.norm(doc1) + LA.norm(doc2)
        return nom/denom

## TF-IDF

In [43]:
def TF_IDF(word, doc, corpus):
    count = 0
    for w in doc:
        if word == w:
            count += 1
    count2 = 0
    for d in corpus:
        if word in d:
            count2 += 1
    TF = count / len(doc)
    IDF = np.log(len(corpus) / (count2 + 1))
    return TF*IDF

## Vanilla K-Means

In [ ]:
def bisectingkmeans(doc_vecs, k): #using global word2vec dictionary loaded above
    clusters = [[doc_vecs]]
    
    
    return None

In [79]:
a = queue.PriorityQueue()
a.put((100,'a'))
a.put((1000,'b'))
print(a.get())

(100, 'a')
